## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from potentiel_solaire.database.queries import get_high_priority_schools
from potentiel_solaire.features.roof_attributes import segmentation_toits, recuperation_mns

from potentiel_solaire.constants import DATA_FOLDER, RESULTS_FOLDER

## Constants

In [ ]:
DEFAULT_CRS_METERS = 6933
sample_size = 500
random_state = 45

## Bâtiments des écoles prioritaires pour Greenpeace

In [ ]:
priotirized_schools = get_high_priority_schools()

priotirized_schools_buildings = []
for code_departement in priotirized_schools["code_departement"].unique():
    # Read results for the current department
    path_results = RESULTS_FOLDER / f"D{code_departement}_pipeline_results.gpkg"
    schools_buildings_dep = gpd.read_file(
        path_results, layer="solar_potential_of_schools_buildings"
    ).to_crs(DEFAULT_CRS_METERS)[[
        "cleabs_bat", 
        "identifiant_de_l_etablissement", 
        "surface_totale_au_sol", 
        "surface_utile", 
        "geometry"
    ]]

    # Get priotirized schools for the current department
    priotirized_schools_dep = priotirized_schools[priotirized_schools["code_departement"] == code_departement]

    # Filter priotirized schools buildings for the current department
    priotirized_schools_buildings_dep = schools_buildings_dep.merge(
        priotirized_schools_dep[["identifiant_de_l_etablissement", "code_departement"]],
        on="identifiant_de_l_etablissement",
        how="inner"
    )

    # Append to the list
    priotirized_schools_buildings.append(priotirized_schools_buildings_dep)


# Concatenate all departements geodataframes
priotirized_schools_buildings = gpd.GeoDataFrame(
    pd.concat(priotirized_schools_buildings, ignore_index=True), 
    crs=DEFAULT_CRS_METERS
)

nb_priotirized_schools = len(priotirized_schools_buildings["identifiant_de_l_etablissement"].unique())
nb_priotirized_schools_buildings = len(priotirized_schools_buildings)
print(f"Nombre d'établissements scolaires prioritaires : {nb_priotirized_schools}")
print(f"Nombre de bâtiments d'établissements scolaires prioritaires : {nb_priotirized_schools_buildings}")

## Proportion de batiments avec ou sans surface utile selon notre algorithme

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]

plt.pie(
    [len(solar_potential_of_schools_buildings[solar_potential_of_schools_buildings["surface_utile"]==0]),
     len(solar_potential_of_schools_buildings[solar_potential_of_schools_buildings["surface_utile"]>0])],
    labels=["Pas de surface utile", "Surface utile disponible"],
    autopct="%1.1f%%",
    startangle=90,                                                                                                                                                  
)

plt.title("Répartition des bâtiments scolaires selon la surface utile disponible")

plt.show()

## Récupération de la surface utile et la pente par batiments grâce aux données MNS

In [ ]:
def get_usable_surface(building):

    print(building["index"], "/", sample_size)

    building = gpd.GeoDataFrame([building.values], columns=solar_potential_of_schools_buildings_sampled.columns, crs=2154)

    mns = segmentation_toits(recuperation_mns(building))

    mns_surface_utile = mns["surface"].sum()

    mns_slope = mns["slope"].to_numpy()

    return pd.Series({
        "mns_surface_utile": mns_surface_utile,
        "mns_slope": mns_slope,
    })

solar_potential_of_schools_buildings_sampled = solar_potential_of_schools_buildings.sample(sample_size, random_state=random_state).reset_index(drop=True)
solar_potential_of_schools_buildings_sampled["index"] = solar_potential_of_schools_buildings_sampled.index

solar_potential_of_schools_buildings_sampled[["mns_surface_utile", "mns_slope"]] = solar_potential_of_schools_buildings_sampled.apply(
    get_usable_surface,
    axis=1,
)

In [ ]:
solar_potential_of_schools_buildings_sampled["mns_slope_mean"] = solar_potential_of_schools_buildings_sampled["mns_slope"].apply(lambda x: x.mean())

## Comparaison de la surface utile calculée par le MNS avec la notre

In [ ]:
# add a label to the points based on the slope and bin the slope into 5 bins
solar_potential_of_schools_buildings_sampled["mns_slope_mean_label"] = pd.cut(
    solar_potential_of_schools_buildings_sampled["mns_slope_mean"],
    bins=5,
    labels=["très faible", "faible", "moyenne", "forte", "très forte"],
)

dict_colors = {
    "très faible": "b",
    "faible": "y",
    "moyenne": "r",
    "forte": "k",
    "très forte": "g",
}

plt.rcParams['figure.figsize'] = [20, 10]

# plot a line y=x
x = [0, solar_potential_of_schools_buildings_sampled["surface_utile"].max()]
y = x
plt.plot(x, y, "r--")

for label, color in dict_colors.items():
    plt.plot(
        solar_potential_of_schools_buildings_sampled[solar_potential_of_schools_buildings_sampled["mns_slope_mean_label"]==label]["surface_utile"],
        solar_potential_of_schools_buildings_sampled[solar_potential_of_schools_buildings_sampled["mns_slope_mean_label"]==label]["mns_surface_utile"],
        "o",
        color=color,
        label=label,
    )

plt.legend(title="Pente moyenne")

plt.xlabel("Surface utile")
plt.ylabel("Surface utile MNS")
plt.title("Surface utile MNS vs surface utile")

plt.show()

In [ ]:
solar_potential_of_schools_buildings_sampled["euclidean_distance"] = abs(solar_potential_of_schools_buildings_sampled["surface_utile"] - solar_potential_of_schools_buildings_sampled["mns_surface_utile"])
solar_potential_of_schools_buildings_sampled["euclidean_distance"].describe()

In [ ]:
solar_potential_of_schools_buildings_sampled["mns_slope_mean"] = solar_potential_of_schools_buildings_sampled["mns_slope"].apply(lambda x: x.mean())

# Distribution de la pente moyenne par batiment

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

plt.hist(
    solar_potential_of_schools_buildings_sampled["mns_slope_mean"],
    bins=5,
)

plt.xlabel("Pente moyenne")
plt.ylabel("Nombre de bâtiments")
plt.title("Distribution des pentes moyennes des bâtiments scolaires échantillonnés")
plt.show()

In [ ]:
solar_potential_of_schools_buildings_sampled.to_csv(
    DATA_FOLDER / f"solar_potential_of_schools_buildings_sampled_{code_departement}.csv",
    index=False,
)